In [1]:
pip install psycopg2-binary pandas openpyxl sqlalchemy redshift_connector tqdm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import redshift_connector
import pandas as pd
from openpyxl import load_workbook, Workbook
from getpass import getpass
import os
from tqdm import tqdm

# Prompt for user credentials
user = input("Enter your username: ")
password = getpass("Enter your password: ")

# Database connection details for Redshift
db_config = {
    'dbname': 'analytics',
    'user': user,
    'password': password,
    'host': '127.0.0.1',
    'port': '17592'  # Default port for Redshift
}

# Connect to Redshift using redshift_connector
conn = redshift_connector.connect(
    host=db_config['host'],
    port=int(db_config['port']),
    database=db_config['dbname'],
    user=db_config['user'],
    password=db_config['password']
)

# List of SQL queries and their corresponding sheet names
queries = {

'NEW_KYC': """
        --NEW KYC
--KYC PASSED USER
WITH base as (
SELECT
last_day( TO_CHAR(DATEADD('months', -number, date_trunc('days',GETDATE())), 'yyyy-mm-dd')::date) AS created_at
FROM ref_numbers
WHERE id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
ORDER BY 1 ASC
)
, v_users as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from workspace.sam_RLU_NU_v1
)
, nokyc_mtd_before_jul AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_mtd
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and date_part('d',registration_date_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')--change the number of days here 
and registration_date_est5edt::date < '2023-07-01'
GROUP BY 1,2
)
, nokyc_mo_before_jul AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_mo
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and registration_date_est5edt::date < '2023-07-01'
GROUP BY 1,2
)
, prod as (
select e.usertoken,
effective_start_datetime,
enrollment_status,
enrollment_timestamp,
convert_timezone('America/Los_Angeles', enrollment_timestamp) as enrollment_timestamp_PDT,
last_day(trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime)))as registration_date,
case when registration_date=last_day(effective_start_datetime) then 'NU' else 'RLU' end as user_type
from prod.sesame_cash.sesame_cash_enrollment e
join prod.public.user u
on e.usertoken = u.usertoken
where effective_end_datetime='2100-01-01 00:00:00'
and enrollment_timestamp is not null
and enrollment_timestamp::date >='2023-01-01'
)
, rlu as (
select *
from
stack.v_users a
left join prod b
on a.partner_user_id=b.usertoken
where created_at_est5edt>='2023-01-01'
and org_name='Sesame Cash'
)
, v_users_new as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from rlu
)
, nokyc_mtd AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_mtd
FROM v_users_new
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and date_part('d',registration_date_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the number of days here 
and registration_date_est5edt::date >= '2023-07-01'
GROUP BY 1,2
)
, nokyc AS (
SELECT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed
FROM v_users_new
WHERE manual_kyc_passed_date_est5edt IS NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
and registration_date_est5edt::date >= '2023-07-01'
AND status = 'REGISTERED'
GROUP BY 1,2
)
, nokyc_mtd_final AS(
SELECT *
FROM nokyc_mtd_before_jul
UNION 
SELECT *
FROM nokyc_mtd
)
, nokyc_final AS(
SELECT *
FROM nokyc_mo_before_jul
UNION 
SELECT *
FROM nokyc
)
SELECT
date_trunc('mon',created_at)::date AS mo
, e.user_type_adjusted
, passed_mtd
, passed_mo
FROM base a
LEFT JOIN nokyc_final e ON a.created_at = e.date 
LEFT JOIN nokyc_mtd_final m ON a.created_at = m.date 
        AND e.user_type_adjusted =  m.user_type_adjusted
ORDER BY 1,2;
    """,


'KYC_Passed_After_FAILING': """
       --KYC PASSED AFTER FAILING USER + MTD
with prod as (
select e.usertoken,
effective_start_datetime,
enrollment_status,
enrollment_timestamp,
convert_timezone('America/Los_Angeles', enrollment_timestamp) as enrollment_timestamp_PDT,
last_day(trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime)))as registration_date,
case when registration_date=last_day(effective_start_datetime) then 'NU' else 'RLU' end as user_type
from prod.sesame_cash.sesame_cash_enrollment e
join prod.public.user u
on e.usertoken = u.usertoken
where effective_end_datetime='2100-01-01 00:00:00'
and enrollment_timestamp is not null
and enrollment_timestamp::date >='2023-01-01'
)
, rlu as (
select *
from
stack.v_users a
left join prod b
on a.partner_user_id=b.usertoken
where created_at_est5edt>='2023-01-01'
and org_name='Sesame Cash'
)
, base as (
SELECT
last_day( TO_CHAR(DATEADD('months', -number, date_trunc('days',GETDATE())), 'yyyy-mm-dd')::date) AS created_at
FROM ref_numbers
WHERE id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
ORDER BY 1 ASC
)
, v_users as (
select *
, case when user_type is null then 'RLU' else user_type end as user_type_adjusted
from rlu
)
, enrolled_mtd AS (
SELECT DISTINCT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_after_failing_mtd
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NOT NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
and date_part('d',registration_date_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the days here
GROUP BY 1,2
ORDER BY 1
)
, enrolled AS (
SELECT DISTINCT
last_day(registration_date_est5edt)::DATE AS date
, user_type_adjusted
, count(1) AS passed_after_failing
FROM v_users
WHERE manual_kyc_passed_date_est5edt IS NOT NULL
AND org_name = 'Sesame Cash'
AND registration_date_est5edt >= '2023/01/01 00:00'
AND status = 'REGISTERED'
GROUP BY 1,2
ORDER BY 1
)
SELECT
date_trunc('mon',created_at)::date AS mo
, e.user_type_adjusted
, passed_after_failing_mtd
, passed_after_failing
FROM base a
LEFT JOIN enrolled e ON a.created_at = e.date 
LEFT JOIN enrolled_mtd m ON a.created_at = m.date 
        AND e.user_type_adjusted =  m.user_type_adjusted
ORDER BY 1,2;
""",
    

'Failed_KYC': """ 
--Failed KYC
WITH base AS (
    SELECT
        DATE_TRUNC('month', DATEADD('months', -number, date_trunc('days', GETDATE())))::date AS created_at
    FROM ref_numbers
    WHERE id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
    ORDER BY 1 ASC
),
user_types AS (
    SELECT 'NU' AS user_type_adjusted
    UNION ALL
    SELECT 'RLU' AS user_type_adjusted
),
all_combinations AS (
    SELECT 
        b.created_at AS mo,
        ut.user_type_adjusted
    FROM 
        base b
    CROSS JOIN 
        user_types ut
),
v_users AS (
    SELECT *
    , COALESCE(user_type, 'RLU') AS user_type_adjusted
    FROM workspace.sam_RLU_NU_v1
),
prod AS (
    SELECT e.usertoken,
           effective_start_datetime,
           enrollment_status,
           enrollment_timestamp,
           convert_timezone('America/Los_Angeles', enrollment_timestamp) AS enrollment_timestamp_PDT,
           DATE_TRUNC('month', trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime))) AS registration_date,
           CASE WHEN registration_date = DATE_TRUNC('month', effective_start_datetime) THEN 'NU' ELSE 'RLU' END AS user_type
    FROM prod.sesame_cash.sesame_cash_enrollment e
    JOIN prod.public.user u ON e.usertoken = u.usertoken
    WHERE effective_end_datetime = '2100-01-01 00:00:00'
      AND enrollment_timestamp IS NOT NULL
      AND enrollment_timestamp::date >= '2023-01-01'
),
rlu AS (
    SELECT *
    FROM stack.v_users a
    LEFT JOIN prod b ON a.partner_user_id = b.usertoken
    WHERE created_at_est5edt >= '2023-01-01'
      AND org_name = 'Sesame Cash'
),
failed_kyc_before_mtd AS (
    SELECT
        DATE_TRUNC('month', created_at_est5edt)::DATE AS date,
        user_type_adjusted,
        COUNT(1) AS failed_kyc_mtd
    FROM v_users
    WHERE org_name = 'Sesame Cash'
      AND manual_kyc_passed_date_est5edt IS NULL
      AND created_at_est5edt::date >= '2023-01-01'
      AND status = 'FAILED_KYC'
      AND date_part('d', created_at_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') -- change the number of days here
      AND created_at_est5edt::date < '2023-08-01'
    GROUP BY 1, 2
    ORDER BY 1
),
failed_kyc_before_mo AS (
    SELECT
        DATE_TRUNC('month', created_at_est5edt)::DATE AS date,
        user_type_adjusted,
        COUNT(1) AS failed_kyc_mo
    FROM v_users
    WHERE org_name = 'Sesame Cash'
      AND manual_kyc_passed_date_est5edt IS NULL
      AND created_at_est5edt::date >= '2023-01-01'
      AND status = 'FAILED_KYC'
      AND created_at_est5edt::date < '2023-08-01'
    GROUP BY 1, 2
    ORDER BY 1
),
v_users_new AS (
    SELECT *
    , COALESCE(user_type, 'RLU') AS user_type_adjusted
    FROM rlu
),
failed_kyc_mtd AS (
    SELECT
        DATE_TRUNC('month', created_at_est5edt)::DATE AS date,
        user_type_adjusted,
        COUNT(1) AS failed_kyc_mtd
    FROM v_users_new
    WHERE org_name = 'Sesame Cash'
      AND manual_kyc_passed_date_est5edt IS NULL
      AND created_at_est5edt::date >= '2023-08-01'
      AND status = 'FAILED_KYC'
      AND date_part('d', created_at_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the days here
    GROUP BY 1, 2
    ORDER BY 1
),
failed_kyc AS (
    SELECT
        DATE_TRUNC('month', created_at_est5edt)::DATE AS date,
        user_type_adjusted,
        COUNT(1) AS failed_kyc
    FROM v_users_new
    WHERE org_name = 'Sesame Cash'
      AND manual_kyc_passed_date_est5edt IS NULL
      AND created_at_est5edt::date >= '2023-08-01'
      AND status = 'FAILED_KYC'
    GROUP BY 1, 2
    ORDER BY 1
),
failed_kyc_mtd_final AS (
    SELECT *
    FROM failed_kyc_before_mtd
    UNION ALL
    SELECT *
    FROM failed_kyc_mtd
),
failed_kyc_final AS (
    SELECT *
    FROM failed_kyc_before_mo
    UNION ALL
    SELECT *
    FROM failed_kyc
)
SELECT
    ac.mo,
    ac.user_type_adjusted,
    COALESCE(m.failed_kyc_mtd, 0) AS failed_kyc_mtd,
    COALESCE(e.failed_kyc_mo, 0) AS failed_kyc_mo
FROM all_combinations ac
LEFT JOIN failed_kyc_final e ON ac.mo = e.date AND ac.user_type_adjusted = e.user_type_adjusted
LEFT JOIN failed_kyc_mtd_final m ON ac.mo = m.date AND ac.user_type_adjusted = m.user_type_adjusted
ORDER BY 1, 2;
""",


'NEW_CB': """
        --NEW CB
with base as (
select 
DATEADD('months', -number, date_trunc('month',GETDATE()))::date AS created_at
from ref_numbers
where id <= DATE_DIFF('months', '2022-12-01', GETDATE()) -- set start date
order by 1 
), cb2_base as (
select
i2c_customer_id
, opt_in_date::date as cb2_opt_date
, status
, created_at_est5edt::date as creation_date
, registration_date_est5edt::date as registration_date
, received_first_fund_date::date as funding_date
, first_funding_date::date as money_hit_account_date
, w.usertoken as pilot_user
, case when registration_date<'2023-08-15' and w.usertoken is null then 'existing Cash Users'
else 'GA enrolled Users' end as cash_status
from analytics.stack.secured_card_settings cb
left join analytics.stack.v_users a
on a.user_id=cb.user_id
left join ssrp_us.v_user_cohort_first_funding f
on f.customer_id=a.i2c_customer_id
left join workspace.sam_CB2_whitelist_all_cohorts_july14 w
on w.usertoken=a.partner_user_id
where cb_version= '2'
and is_opted_in='TRUE'
), cb2_funded_monthly as (
select date_trunc('mon',local_date_time)::date 
, count(distinct a.customer_id) as cb2_funded
from stack.v_event_logs_i2c_alert_incoming a
inner join cb2_base b
on a.customer_id=b.i2c_customer_id
left join ssrp_us.v_user_cohort_first_funding f
on a.customer_id = f.customer_id
where local_date_time::date >= '2023-07-01'
and card_program_id='CS Secured M CB2'
and first_funding_month >= '2023-08-31'
and service='Activate Card'
group by 1
), cb2_funded_mtd as (
select date_trunc('mon',local_date_time)::date 
, count(distinct a.customer_id) as cb2_funded_mtd
from stack.v_event_logs_i2c_alert_incoming a
inner join cb2_base b
on a.customer_id=b.i2c_customer_id
left join ssrp_us.v_user_cohort_first_funding f
on a.customer_id = f.customer_id
where local_date_time::date >= '2023-07-01'
and card_program_id='CS Secured M CB2'
and first_funding_month >= '2023-08-31'
and date_part('d',local_date_time) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') -- change the number of days here
and service='Activate Card'
group by 1
), mtd as(
select date_trunc('mon',first_credit_builder_month)::date
, sum(case when b.customer_id is not null then 1 else 0 end) as new_funded_cb_users_mtd
from ssrp_us.v_user_cohort_credit_builder a
left join ssrp_us.v_user_cohort_first_funding b
on a.customer_id=b.customer_id
and a.first_credit_builder_month=b.first_funding_month
where date_part('d', first_credit_builder_date) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')-- change the number of days here
and first_credit_builder_month >= '2023-01-01'
group by 1
order by 1
), monthly as (
select date_trunc('mon',first_credit_builder_month)::date
, sum(case when b.customer_id is not null then 1 else 0 end) as new_funded_cb_users
from ssrp_us.v_user_cohort_credit_builder a
left join ssrp_us.v_user_cohort_first_funding b
on a.customer_id=b.customer_id
and a.first_credit_builder_month=b.first_funding_month
where first_credit_builder_month >= '2023-01-01'
group by 1
order by 1
), cb_enroll as (
select
 date_trunc('mon',opt_in_date::date) as opt_in_mo
, SUM(CASE WHEN cb_version='2' and received_first_fund='TRUE' THEN 1 END) AS cb2_funded_enrolled
, SUM(CASE WHEN cb_version='2' and received_first_fund='TRUE' and date_part ('d',opt_in_date::date)<= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') THEN 1 END) AS cb2_funded_enrolled_mtd -- change the number of days here
from stack.v_users a
left join analytics.stack.secured_card_settings cb
on a.user_id=cb.user_id
left join ssrp_us.v_user_cohort_first_funding f
on f.customer_id=a.i2c_customer_id
where  org_name='Sesame Cash'
and is_opted_in='TRUE'
group by 1
order by 1
)
select b.created_at
,nvl(mtd.new_funded_cb_users_mtd,0) + nvl(cb2_funded_mtd,0) as new_funded_cb_mtd
,cb2_funded_enrolled_mtd
,nvl(m.new_funded_cb_users,0) + nvl(cb2_funded,0) as new_funded_cb_monthly
,cb2_funded_enrolled
from base b
left join monthly m on b.created_at = m.date_trunc
left join mtd on b.created_at = mtd.date_trunc
left join cb2_funded_monthly on b.created_at = cb2_funded_monthly.date_trunc
left join cb2_funded_mtd on b.created_at = cb2_funded_mtd.date_trunc
left join cb_enroll on b.created_at = cb_enroll.opt_in_mo
order by 1
;""",


'CB_Churn': """
--CB Churn 
with history as(
select secured_card_setting_id
,created_at
,row_number() over (partition by secured_card_setting_id order by created_at desc) as rn
from stack.secured_card_settings_history
where is_opted_in = 'FALSE'
and changes not like '%cbVersion%'
),cb as(
select s.user_id
,h.created_at
from history h
left join stack.secured_card_settings s
on h.secured_card_setting_id = s.secured_card_setting_id
where rn =1
and s.is_opted_in = 'FALSE'
),base as (
SELECT
b.*
,last_day(cb.created_at) as CLOSE_DATE
FROM ssrp_us.v_user_cohort_credit_builder b
INNER JOIN stack.v_users a ON (a.i2c_customer_id = b.customer_id)
LEFT JOIN cb ON (a.user_id = cb.user_id)
where date_part('d',created_at) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the number of days
), mtd as(
select 
date_trunc('mon',close_date)::date
, count(1)
from base
where close_date is not null
and close_date >= '2023-01-01'
group by 1
order by 1
),base_1 as (
SELECT
b.*
,last_day(cb.created_at) as CLOSE_DATE
FROM ssrp_us.v_user_cohort_credit_builder b
INNER JOIN stack.v_users a ON (a.i2c_customer_id = b.customer_id)
LEFT JOIN cb ON (a.user_id = cb.user_id)
), monthly as(
select 
date_trunc('mon',close_date)::date
, count(1)
from base_1
where close_date is not null
and close_date >= '2023-01-01'
group by 1
order by 1
)
select monthly.date_trunc
,monthly.count as count_monthly
,mtd.count as count_mtd
from mtd
join monthly
on mtd.date_trunc = monthly.date_trunc
order by 1
;
""",

#Need to update the date in this each month
'InactiveandFunded': """--inactive with and without funds
with active as (
select f.customer_id
, min(f.transaction_datetime::DATE) as initial_active_date30
, last_day(f.transaction_datetime) as active_month30
from finance.sesame_reconciliation f
inner join stack.v_users u
on f.customer_id = u.i2c_customer_id
where u.org_name = 'Sesame Cash'
and f.active is true
group by 1,3
order by 1,3
), active_base as (
select customer_id
,active_month30
from active
where active_month30 = '2024-08-31' --change month here 
), fund_status as (
select f.customer_id
,v.status
,sum(ledger_balance) as balance
from stack.v_users v
left join ssrp_us.raw_nf r
on v.i2c_customer_id = r.customer_id
inner join ssrp_us.v_user_cohort_first_funding f
on v.i2c_customer_id = f.customer_id
where imported_at::date = '2024-08-31'  --change month here
and org_name = 'Sesame Cash'
and program_identification_value = 'Sesame Cash Prepaid'
group by 1,2
order by 1
), inactive_fund_status as(
select f.customer_id
,balance
,case when balance >0 then 'with fund' else 'no fund' end as fund_status
,active_month30
from fund_status f
left join active_base a
on f.customer_id= a.customer_id
where active_month30 is null
)
select fund_status
,count(distinct customer_id)
from inactive_fund_status
group by 1
;""",


'New_Account_Closure': """--NEW ACCOUNT CLOSURE
with base as(
select i2c_customer_id
,status
,updated_at_est5edt
,row_number() over (partition by i2c_customer_id order by updated_at_est5edt desc) as rn
from stack.v_users v
inner join ssrp_us.v_user_cohort_first_funding f
on v.i2c_customer_id = f.customer_id
where org_name = 'Sesame Cash'
and status = 'DELETED'
order by 1
), mtd as (
select date_trunc('mon',updated_at_est5edt)::date as tx_mo
,count(distinct i2c_customer_id)
from base
where date_part('d',updated_at_est5edt) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the number of days
and date_trunc('mon',updated_at_est5edt)::date >= '2023-01-01'
group by 1
order by 1
), monthly as (
select date_trunc('mon',updated_at_est5edt)::date as tx_mo
,count(distinct i2c_customer_id)
from base
where date_trunc('mon',updated_at_est5edt)::date >= '2023-01-01'
group by 1
order by 1
)
select monthly.tx_mo
,monthly.count as monthly_count
,mtd.count as mtd_count
from monthly
join mtd
on monthly.tx_mo = mtd.tx_mo
order by 1
;
 """,


'Cash_Fee_Waived': """--CASH FEE WAIVED
with fee_waived_mo as (
select customer_id
,transaction_description
,min(transaction_datetime)::date
,last_day(transaction_datetime)::date as waived_mo
from stack.v_event_logs_i2c_alert_incoming
where transaction_description not in ('Inactivity Fee', 'Sesame Cash Monthly Service Fee')
and card_program_id='Sesame Cash Prepaid'
and transaction_datetime::date >= '2023-06-01'
group by 1,2,4
order by 1
), pos_over_1k as (
select customer_id
,last_day(date_trunc('mon',transaction_datetime))::date as pos_mo
,sum(transaction_amount)* -1 as pos
from finance.sesame_reconciliation
where finance_classification = 'POS' 
and card_program_id='Sesame Cash Prepaid'
and transaction_datetime::date >= '2023-06-01'
group by 1,2
having pos >= 1000
order by 1
),dd_over_500 as (
select f.customer_id
, last_day(date_trunc('mon',f.transaction_datetime))::DATE as dd_mo
, SUM(f.transaction_amount) as tx_amount
from finance.sesame_reconciliation f
inner join stack.v_users u 
on f.customer_id = u.i2c_customer_id
where u.org_name = 'Sesame Cash'
and u.status <> 'FRAUD_BLOCK'
and f.transaction_classification IN ('Direct Deposit','Direct Deposit Reversal','Direct Deposit Adjustment')
and f.transaction_classification_group <> 'Money Load (Micro Deposit)'
and date_trunc('mon',f.transaction_datetime) >= '2023-06-01'
group by 1,2
having tx_amount >= 500
order by 1,2
),active_mo as (
select customer_id
,min(transaction_datetime)::date
,last_day(transaction_datetime)::date as active_mo
from finance.sesame_reconciliation
where transaction_datetime::date >= '2023-06-01'
and active is true 
group by 1,3
order by 1
)
select 
waived_mo
,count(distinct f.customer_id) 
from fee_waived_mo f
left join pos_over_1k p
on f.customer_id = p.customer_id and f.waived_mo = pos_mo
left join dd_over_500 d
on f.customer_id = d.customer_id and f.waived_mo = dd_mo
left join active_mo a
on f.customer_id = a.customer_id and f.waived_mo = active_mo
where active_mo is not null 
and (pos_mo is not null or dd_mo is not null)
group by 1
order by 1
;
 """,


'Active': """ --ACTIVE
select date_trunc('mon',transaction_datetime)::date as active_month30
,count(distinct customer_id) as monthly_count
-- change the number of days below
,count(distinct case when date_part('d',transaction_datetime) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') then customer_id end) as monthly_count -- change the number of days  
from finance.sesame_reconciliation
where active IS TRUE
and transaction_datetime::DATE >= '2023-01-01'
and transaction_classification_group not in ('Credit Booster','Credit Builder 2')
group by 1
order by 1
;""",


'Banking_POS': """ --Banking POS
with pos_mtd as (
SELECT
DATE_TRUNC('mon',f.transaction_datetime)::DATE AS transaction_mo
, SUM(CASE WHEN f.finance_classification = 'POS' THEN transaction_amount END) * -1 AS banking_spending_amount
FROM finance.sesame_reconciliation f
INNER JOIN stack.v_users u ON f.customer_id = u.i2c_customer_id
WHERE u.org_name = 'Sesame Cash'
AND u.status <> 'FRAUD_BLOCK'
AND f.transaction_datetime::DATE >= '2023-01-01'
AND DATE_PART('d',transaction_datetime) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')  --change the number of days 
GROUP BY 1
ORDER BY 1
), pos_monthly as (
SELECT
DATE_TRUNC('mon',f.transaction_datetime)::DATE AS transaction_mo
, SUM(CASE WHEN f.finance_classification = 'POS' THEN transaction_amount END) * -1 AS banking_spending_amount
FROM finance.sesame_reconciliation f
INNER JOIN stack.v_users u ON f.customer_id = u.i2c_customer_id
WHERE u.org_name = 'Sesame Cash'
AND u.status <> 'FRAUD_BLOCK'
AND f.transaction_datetime::DATE >= '2023-01-01'
GROUP BY 1
ORDER BY 1
)
select mo.transaction_mo
,mo.banking_spending_amount as monthly_banking_spending_amount
,m.banking_spending_amount as mtd_banking_spending_amount
from pos_monthly mo
join pos_mtd m
on mo.transaction_mo = m.transaction_mo
order by 1
;
""",


'Cash_Fee': """ --CASH FEE
with base as (
select transaction_datetime::date as tx_date
, customer_id
,3 as unit_1
,9.99 as unit_2
,service
,case
when transaction_description = 'Inactivity Fee' and ledger_balance >= 0 then unit_1
when transaction_description = 'Inactivity Fee' and ledger_balance between -3 and 0 then (unit_1+ledger_balance)
when transaction_description = 'Sesame Cash Monthly Service Fee' and ledger_balance >= 0 then unit_2
when transaction_description = 'Sesame Cash Monthly Service Fee' and ledger_balance between -9.99 and 0 then (unit_2+ledger_balance)
else 0
end as cash_fee
, ledger_balance
from stack.v_event_logs_i2c_alert_incoming
where transaction_description in ('Inactivity Fee', 'Sesame Cash Monthly Service Fee')
and card_program_id='Sesame Cash Prepaid'
and transaction_datetime::date >= '2023-06-01'
), revenue as (
select tx_date
, b.customer_id
, service
, case when service='Account Maintenance' then 9.99 else 3 end as charge
, ledger_balance as ledger_balance_after_charge
, cash_fee
, case when last_day(tx_date) = a.active_month30 then 1 else 0 end as whether_active
from base b
left join ssrp_us.v_user_cohort_active30 a
on b.customer_id = a.customer_id
and last_day(tx_date) = a.active_month30 
)
select tx_date
, whether_active
--, sum(case when service = 'Account Maintenance' then charge end) as cash_charge
--, sum(case when service = 'Card Inactivity' then charge end) as inactivity_charge
, sum(case when service = 'Account Maintenance' then cash_fee end) as cash_fee
, sum(case when service = 'Card Inactivity' then cash_fee end) as inactivity_fee
--, count(distinct case when service = 'Account Maintenance' then customer_id end) as cash_fee_count
--, count(distinct case when service = 'Card Inactivity' then customer_id end) as inactivity_fee_count
from revenue
group by 1,2
order by 1,2
;
 """,


'User_refunded_track': """--user refund track 1.0
with cash_fee_base as (
--grab fee charge amount and charge date for each customer
select
customer_id
,transaction_description
,transaction_datetime as fee_charge_time
,ledger_balance as balance_after_fee
,abs(transaction_amount) as fee_charged
,case when ledger_balance >=0 then 0
when (ledger_balance < 0 and abs(ledger_balance) >= abs(transaction_amount)) then abs(transaction_amount)
when (ledger_balance < 0 and abs(ledger_balance) < abs(transaction_amount)) then abs(ledger_balance)
end as fee_not_recognized
,row_number () over (partition by customer_id order by transaction_datetime desc) as rn_tx
,row_number () over (partition by customer_id order by ledger_balance) as rn_balance
from stack.v_event_logs_i2c_alert_incoming
where transaction_description in ('Inactivity Fee', 'Sesame Cash Monthly Service Fee')
and card_program_id='Sesame Cash Prepaid'
and transaction_datetime::date >= '2023-06-01'
and ledger_balance < 0
--and customer_id in ('388000000002122786','388000000002357025')
order by 1
), cash_fee_1 as(
--cash_fee_1 calculate the total_fee_not_recognized to each cx that fund after 2 charge dates
select customer_id
,transaction_description
,sum(fee_not_recognized) as total_fee_not_recognized
from cash_fee_base
where rn_tx = rn_balance
--and customer_id = '388000000000000698'
group by 1,2
order by 1
), load_after_fee_charged as(
select c.customer_id
,c.total_fee_not_recognized
,c.transaction_description
,b.fee_charge_time
,v.transaction_datetime as load_date
,v.transaction_amount as load_amount
,v.transaction_classification
--,row_number () over (partition by c.customer_id order by v.transaction_datetime) as load_rn
, row_number () over (partition by c.customer_id order by case when c.transaction_description = 'Sesame Cash Monthly Service Fee'
then v.transaction_datetime end) as load_rn_service
, row_number () over (partition by c.customer_id order by case when c.transaction_description = 'Inactivity Fee'
then v.transaction_datetime end) as load_rn_inactive
from cash_fee_1 c
left join cash_fee_base b
on c.customer_id = b.customer_id
left join ssrp_us.v_posted_transactions v
on c.customer_id = v.customer_id
where b.rn_tx =1
and (transaction_classification_group = 'Money Load' or transaction_classification = 'CB Funds Withdrawal +')
and b.fee_charge_time <= v.transaction_datetime
--and c.customer_id = '388000000000000698'
--and c.customer_id = '388000000000000593'
--and c.customer_id = '388000000000000954'
order by 1
), rev_recognized_1 as (
--capture the revenue recognized by the user funded after two charge dates
select customer_id
,fee_charge_time
,transaction_description
,load_date
,total_fee_not_recognized
,load_amount
,case when load_amount >= total_fee_not_recognized then total_fee_not_recognized
when load_amount < total_fee_not_recognized then load_amount
end as rev_recognized
from load_after_fee_charged
where --load_rn = 1
(load_rn_service =1 or load_rn_inactive =1)
--and customer_id = '388000000000902893'
order by 1
), cash_fee_2 as(
--cash_fee_2 calculate the total charge to each cx that fund in between the 2 charge date
select c.customer_id
,c.transaction_description
,c.fee_charge_time
,c.balance_after_fee
,c.fee_not_recognized
,v.transaction_datetime as load_date
,v.transaction_amount as load_amount
,v.transaction_classification
,row_number () over (partition by c.customer_id order by v.transaction_datetime) as load_rn
from cash_fee_base c
left join ssrp_us.v_posted_transactions v
on c.customer_id = v.customer_id
where (transaction_classification = 'CB Funds Withdrawal +' or transaction_classification_group = 'Money Load' )
and c.fee_charge_time <= v.transaction_datetime
and c.rn_tx <> c.rn_balance
order by 1
),rev_recognized_2 as(
--capture the revenue recognized by the user funded in between two charge dates
select customer_id
,fee_charge_time
,transaction_description
,load_date
,fee_not_recognized
,load_amount
,case when load_amount >= fee_not_recognized and load_rn <=2 then fee_not_recognized
when load_amount < fee_not_recognized and load_rn <=2 then load_amount
end as rev_recognized
from cash_fee_2 c
where rev_recognized is not null
--and customer_id = '388000000002758275'
order by 1
), final as (
select *
from rev_recognized_1
union
select *
from rev_recognized_2
)
select load_date::date
,transaction_description
,count (distinct f.customer_id)
,sum (rev_recognized)
from final f
group by 1,2
order by 1,2
;
 """,


'Rev_generating_user': """ --rev generating user
with base as (
(select
EXTRACT(YEAR FROM transaction_datetime) AS year
,EXTRACT(MONTH FROM transaction_datetime) AS month
,EXTRACT(DAY FROM transaction_datetime) AS day
,customer_id
from ssrp_us.v_posted_transactions
where (interchange_fee_amount > 0
or transaction_classification = 'Cash Withdrawal Fee') --ATM USER and internchange user
and program_identification_value IN ('Sesame Cash NP','Sesame Cash Prepaid')
and transaction_datetime::date >= '2023-02-01'
group by year,month,day,customer_id
order by year,month)
union
(select
EXTRACT(YEAR FROM f.transaction_datetime) AS year
,EXTRACT(MONTH FROM f.transaction_datetime) AS month
,EXTRACT(DAY FROM f.transaction_datetime) AS day
,f.customer_id
from ssrp_us.v_posted_transactions f
left join stack.v_event_logs_i2c_alert_incoming i --Cash FEE i
on f.customer_id = i.customer_id and f.transaction_amount = i.transaction_amount and f.trace_audit_number = i.transaction_id
where i.card_program_id IN ('Sesame Cash NP','Sesame Cash Prepaid')
and f.transaction_classification = 'Sesame Cash Fee'
and f.transaction_amount<=-9.99
and ledger_balance > -9.99
and f.transaction_datetime::date >= '2023-02-01'
group by year,month,day,f.customer_id
order by year,month)
union
(select
EXTRACT(YEAR FROM f.transaction_datetime) AS year
,EXTRACT(MONTH FROM f.transaction_datetime) AS month
,EXTRACT(DAY FROM f.transaction_datetime) AS day
,f.customer_id
from ssrp_us.v_posted_transactions f
left join stack.v_event_logs_i2c_alert_incoming i--Cash FEE i
on f.customer_id = i.customer_id and f.transaction_amount = i.transaction_amount and f.trace_audit_number = i.transaction_id
where i.card_program_id IN ('Sesame Cash NP','Sesame Cash Prepaid')
and f.transaction_classification = 'Inactivity Fee'
and f.transaction_amount<=-3
and ledger_balance > -3
and f.transaction_datetime::date >= '2023-02-01'
group by year,month,day,f.customer_id
order by year,month))
, migration_base as (
select distinct f.customer_id
from ssrp_us.v_posted_transactions f
where (transaction_datetime::date = '2023-11-27' or transaction_datetime::date = '2024-01-25')
and card_program_id = 'CS Security Deposit'
and description = 'Credit Builder Deposit Withdrawal'
order by transaction_datetime) -- grab all the people's id who have migration happen
,previous_balance as
(
select i.*,lag(ledger_balance) over (order by i.customer_id, i.transaction_datetime ASC) as previous_balance
from stack.v_event_logs_i2c_alert_incoming i
inner join migration_base m
on m.customer_id = i.customer_id
where card_program_id = 'Sesame Cash Prepaid'
and ledger_balance is not null)
,rev_migration_list as (
select *
from previous_balance
where
service = 'SpecialCredit6'
and transaction_description = 'Credit Builder Deposit Withdrawal'
and (transaction_datetime::date = '2023-11-27' or transaction_datetime::date = '2024-01-25')
and previous_balance < 0)
, migration_rev_user as (
select
EXTRACT(YEAR FROM transaction_datetime) AS year
,EXTRACT(MONTH FROM transaction_datetime) AS month
,EXTRACT(DAY FROM transaction_datetime) AS day
, customer_id
from rev_migration_list
group by year,month,day,customer_id
order by year,month)
select
year as rev_year,
month as rev_month,
COUNT(distinct customer_id) as rev_generating_user
from ( select * from base union select * from migration_rev_user) as regroup
where day <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')
GROUP BY year, month
ORDER BY year, month;""",


'Rev_generating_user_inactivityfee': """--rev generating user (inactivity fee)
with base as
(select
EXTRACT(YEAR FROM f.transaction_datetime) AS year
,EXTRACT(MONTH FROM f.transaction_datetime) AS month
,EXTRACT(DAY FROM f.transaction_datetime) AS day
,f.customer_id
from ssrp_us.v_posted_transactions f
left join stack.v_event_logs_i2c_alert_incoming i--Cash FEE i
on f.customer_id = i.customer_id and f.transaction_amount = i.transaction_amount and f.trace_audit_number = i.transaction_id
where i.card_program_id IN ('Sesame Cash NP','Sesame Cash Prepaid')
and f.transaction_classification = 'Inactivity Fee'
and f.transaction_amount<=-3
and ledger_balance > -3
and f.transaction_datetime::date >= '2023-02-01'
group by year,month,day,f.customer_id
order by year,month)
select
year as rev_year
,month as rev_month
,count(distinct customer_id) as rev_generating_inactivity_fee_mtd
from base
where day <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')
group by year,month
order by year,month; """,


'Rev_generating_user_cashfeefee': """ --rev generating user (cash fee fee)
with base as
(select
EXTRACT(YEAR FROM f.transaction_datetime) AS year
,EXTRACT(MONTH FROM f.transaction_datetime) AS month
,EXTRACT(DAY FROM f.transaction_datetime) AS day
,f.customer_id
from ssrp_us.v_posted_transactions f
left join stack.v_event_logs_i2c_alert_incoming i--Cash FEE i
on f.customer_id = i.customer_id and f.transaction_amount = i.transaction_amount and f.trace_audit_number = i.transaction_id
where i.card_program_id IN ('Sesame Cash NP','Sesame Cash Prepaid')
and f.transaction_classification = 'Sesame Cash Fee'
and f.transaction_amount<=-9.99
and ledger_balance > -9.99
and f.transaction_datetime::date >= '2023-02-01'
group by year,month,day,f.customer_id
order by year,month)
select
year as rev_year
,month as rev_month
,count(distinct customer_id) as rev_generating_cash_fee_mtd
from base
where day <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day')
group by year,month
order by year,month; """,
}


# Fetch data using the provided query
def fetch_data(query):
    with conn.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(result, columns=columns)
    return df

excel_file_path = 'CashOps-Test.xlsx'

if os.path.exists(excel_file_path):
    os.remove(excel_file_path)

if not os.path.exists(excel_file_path):
    # Create a new Excel file if it does not exist
    wb = Workbook()
    wb.save(excel_file_path)


try:
    book = load_workbook(excel_file_path)
except Exception as e:
    print(f"Error loading workbook: {e}")
    book = Workbook()
    book.save(excel_file_path)
    book = load_workbook(excel_file_path)

with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    for sheet_name, query in tqdm(queries.items(), desc="Running Queries"):
        data = fetch_data(query)
        
        if sheet_name in book.sheetnames:
        
            del book[sheet_name]
        
        data.to_excel(writer, sheet_name=sheet_name, index=False)

print("Data successfully written to Excel file.")

conn.close()


Running Queries:  87%|████████▋ | 13/15 [05:16<00:38, 19.23s/it]/Users/jgonsalves/Library/Python/3.9/lib/python/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
Running Queries: 100%|██████████| 15/15 [05:19<00:00, 21.29s/it]

Data successfully written to Excel file.


In [5]:
import redshift_connector
import pandas as pd
from openpyxl import load_workbook, Workbook
from tqdm import tqdm

db_config2 = {
    'dbname': 'prod',
    'user': user,
    'password': password,
    'host': '127.0.0.1',
    'port': '17592'  
}

try:
    conn2 = redshift_connector.connect(
    
        host=db_config2['host'],
        port=int(db_config2['port']),
        database=db_config2['dbname'],
        user=db_config2['user'],
        password=db_config2['password']
    )
except Exception as e:
    print(f"Error connecting to the database: {e}")
    conn2 = None
queries2 = {

'Cash_Page_View': """--Cash Page View 

with base as (
SELECT
distinct
v.usertoken
,last_day(action_timestamp) as month_view
, last_day(u.acct_registration_complete_datetime) as credit_registration
, last_day(enrollment_timestamp) as banking_enrollement
, enrollment_status as banking_status
, case when credit_registration=month_view then 'NU' else 'RLU' end as user_type
, case when banking_enrollement>=month_view then 1 else 0 end as bank_enrolled_before
FROM public.user_activity v
INNER JOIN public.user u ON v.usertoken = u.usertoken
left join sesame_cash.sesame_cash_enrollment b
on v.usertoken=b.usertoken
and effective_end_datetime='2100-01-01 00:00:00'
WHERE v.session_id IS NOT NULL
AND v.action_type = 'VIEW_PAGE'
and page_name in ('SESAME_CASH_OVERVIEW' , 'SESAME_CASH_OVERVIEW_(UNENROLLED_CASH)'
,'SESAME_CASH_OVERVIEW_(UNENROLLED_CB2.0)', 'CASH')
AND month_view>='2023-01-31'
AND date_part('d',action_timestamp) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the number of days here
), mtd as (
select
month_view
, user_type
, count(1)
from base
where bank_enrolled_before=1 or banking_enrollement is null
group by 1,2
), base_1 as (
SELECT
distinct
v.usertoken
,last_day(action_timestamp) as month_view
, last_day(u.acct_registration_complete_datetime) as credit_registration
, last_day(enrollment_timestamp) as banking_enrollement
, enrollment_status as banking_status
, case when credit_registration=month_view then 'NU' else 'RLU' end as user_type
, case when banking_enrollement>=month_view then 1 else 0 end as bank_enrolled_before
FROM public.user_activity v
INNER JOIN public.user u ON v.usertoken = u.usertoken
left join sesame_cash.sesame_cash_enrollment b
on v.usertoken=b.usertoken
and effective_end_datetime='2100-01-01 00:00:00'
WHERE v.session_id IS NOT NULL
AND v.action_type = 'VIEW_PAGE'
and page_name in ('SESAME_CASH_OVERVIEW' , 'SESAME_CASH_OVERVIEW_(UNENROLLED_CASH)'
,'SESAME_CASH_OVERVIEW_(UNENROLLED_CB2.0)', 'CASH')
AND month_view>='2023-01-31'
), monthly as (
select
month_view
, user_type
, count(1)
from base_1
where bank_enrolled_before=1 or banking_enrollement is null
group by 1,2
)
select 
date_trunc('mon',monthly.month_view)::date,
monthly.user_type,
monthly.count as monthly_count,
mtd.count as mtd_count
from monthly
join mtd
on monthly.month_view = mtd.month_view
and monthly.user_type = mtd.user_type
order by 1,2
;""",


'New_Enrollment': """--NEW ENROLLMENT

with base as (
select e.usertoken,
effective_start_datetime,
enrollment_status,
enrollment_timestamp,
convert_timezone('America/Los_Angeles', enrollment_timestamp) as enrollment_timestamp_PDT,
trunc(convert_timezone('America/Los_Angeles', acct_registration_complete_datetime)) as registration_date,
row_number() over (partition by e.usertoken, enrollment_status order by effective_start_datetime desc) as status_rank
from sesame_cash.sesame_cash_enrollment e
join public.user u
on e.usertoken = u.usertoken
where effective_end_datetime='2100-01-01 00:00:00'
and enrollment_timestamp is not null
and enrollment_timestamp>'2023-01-01'
), mtd as(
select
date_trunc('mon',convert_timezone('America/Los_Angeles', coalesce(enrollment_timestamp, effective_start_datetime)))::date as date,
case
when date_part('month', date) = date_part('month', r.registration_date)
and date_part('year', date) = date_part('year', r.registration_date) then 'NU'
else 'RLU'
end as user_status,
count(distinct usertoken)
from base r
where date_part('d',enrollment_timestamp) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') -- change the number here 
group by 1,2
order by 1,2
), monthly as (
select
date_trunc('mon',convert_timezone('America/Los_Angeles', coalesce(enrollment_timestamp, effective_start_datetime)))::date as date,
case
when date_part('month', date) = date_part('month', r.registration_date)
and date_part('year', date) = date_part('year', r.registration_date) then 'NU'
else 'RLU'
end as user_status,
count(distinct usertoken)
from base r
group by 1,2
order by 1,2
)
select monthly.date,
monthly.user_status,
monthly.count as monthly_count,
mtd.count as mtd_count
from mtd
join monthly
on mtd.date = monthly.date
and mtd.user_status = monthly.user_status
order by 1,2
;
 """,


'New_Funded': """
--NEW FUNDED
with banking as (
    Select
        i2c_customer_id as customer_id,
        registered_datetime::date as platform_date,
        registered_month,
        banking_registered_datetime::date as banking_date,
        banking_registered_month,
        b.first_funding_date::date as funding_date,
        banking_date - platform_date as days_on_platform,
        funding_date - platform_date as days_to_fund
    from one_offs.tc_banking_user_cohort a
    inner join analytics.ssrp_us.v_user_cohort_first_funding b 
    on a.i2c_customer_id = b.customer_id
    where funding_date >= '2022-01-01'
)
, mtd as (
    select
        customer_id,
        last_day(platform_date) as platform_month,
        banking_registered_month,
        last_day(funding_date) as funding_month,
        case when days_on_platform < 0 then 0 else days_on_platform end as days_on_platform,
        case when days_to_fund < 0 then 0 else days_to_fund end as days_to_fund,
        case 
            when funding_month = banking_registered_month 
                and banking_registered_month = platform_month 
            then 'NU'
            when funding_month = banking_registered_month 
            then 'RLU'
            when DATEDIFF(months, banking_registered_month, funding_month) <= 6 
            then 'RLU_1'
            else 'RLU_1'
        end as funded_category
    from banking
    where date_part('d', funding_date) <= EXTRACT(day FROM CURRENT_DATE - INTERVAL '3 day') --change the number of days here
)
, mtd_result as (
    select 
        date_trunc('mon', funding_month)::date as date_trunc,
        funded_category,
        count(distinct customer_id) as mtd_count
    from mtd
    group by 1, 2
    order by 1, 2
)
, monthly as (
    select
        customer_id,
        last_day(platform_date) as platform_month,
        banking_registered_month,
        last_day(funding_date) as funding_month,
        case when days_on_platform < 0 then 0 else days_on_platform end as days_on_platform,
        case when days_to_fund < 0 then 0 else days_to_fund end as days_to_fund,
        case 
            when funding_month = banking_registered_month 
                and banking_registered_month = platform_month 
            then 'NU'
            when funding_month = banking_registered_month 
            then 'RLU'
            when DATEDIFF(months, banking_registered_month, funding_month) <= 6 
            then 'RLU_1'
            else 'RLU_1'
        end as funded_category
    from banking
)
, monthly_result as (
    select
        date_trunc('mon', monthly.funding_month)::date as date_trunc,
        funded_category,
        count(distinct customer_id) as monthly_count
    from monthly
    group by 1, 2
    order by 1, 2
)
select
    o.date_trunc,
    o.funded_category,
    monthly_count,
    mtd_count
from monthly_result o
full join mtd_result t
on o.date_trunc = t.date_trunc
and o.funded_category = t.funded_category
--where o.date_trunc >= '2023-01-01'
order by 1, 2;
"""
,

}

def fetch_data2(query):
    with conn2.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(result, columns=columns)
    return df

if conn2:
    try:
        # Load the Excel workbook
        try:
            book = load_workbook(excel_file_path)
        except Exception as e:
            print(f"Error loading workbook: {e}")
            book = Workbook()
            book.save(excel_file_path)
            book = load_workbook(excel_file_path)

        # Run queries and write data to Excel
        with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            for sheet_name, query in tqdm(queries2.items(), desc="Running Queries"):
                data = fetch_data2(query)
                
                if sheet_name in book.sheetnames:
                    del book[sheet_name]
                
                data.to_excel(writer, sheet_name=sheet_name, index=False)
        print("Data successfully written to Excel file.")
    finally:
        conn2.close()
else:
    print("Failed to connect to the database; exiting script.")



Running Queries:   0%|          | 0/3 [00:00<?, ?it/s]